In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# imports 
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk

import nibabel as nib
from totalsegmentator.python_api import totalsegmentator

import radipop_utils 
import radipop_utils.visualization
import radipop_utils.features
import radipop_utils.utils

from radipop_utils.features import (
    convert_and_extract_from_nii,
)


# load user/ system specific env variables:
from dotenv import dotenv_values, find_dotenv
config = dotenv_values(find_dotenv())  # load environment variables as dictionary

# path to the data. You will (likely need to change this)
DATA_ROOT_DIRECTORY = Path(config["DATA_ROOT_DIRECTORY"])


# needed for loading the settings for the radiomics feature extractor 
path = Path(os.path.abspath(radipop_utils.__file__))
RADIPOP_PACKAGE_ROOT = path.parent.parent



# plan: 
    # convert dicom to nifti (optional step lateron)
    # run the totalsegmentator on the nifti
    # save masks  as nifti

    # load trained scalar prediciton model
    # run on the nifti (image + masks)


In [ ]:
# convert from dictom to nifti
sample_dicom_path = Path("/home/cwatzenboeck/data/project_segmenation_ls/2024_muw_additional_DICOMs/V.313 horos")
tmp_wd_path  = Path("/home/cwatzenboeck/tmp/radipop_tmp_wd")
subfolder_name = "patient_dev"

radipop_utils.utils.dcm2nii(sample_dicom_path, tmp_wd_path, out_id = subfolder_name)



In [ ]:
if __name__ == "__main__":
    input_path = tmp_wd_path / subfolder_name / "base.nii.gz"
    output_path = tmp_wd_path / subfolder_name / "mask_ts.nii.gz"
    input_img = nib.load(input_path)
    output_img = totalsegmentator(input_img)
    nib.save(output_img, output_path)
    
    

In [ ]:
# # load image and plot overlayed masks + image for one slice:
image_loc = input_path
mask_loc = output_path

mask = sitk.ReadImage(output_path)
#img = sitk.ReadImage(image_loc)
img = convert_and_extract_from_nii(image_loc, out_range=[0, 1.0], wl=50, ww=500, dtype=np.float64)  

img_array = sitk.GetArrayFromImage(img)
mask_array = sitk.GetArrayFromImage(mask)
print(img_array.shape, mask_array.shape)


In [ ]:

idx = 300
image_xy_slice = img_array[idx, :, :]
label_xy_slice = mask_array[idx, :, :]

crop = lambda x: radipop_utils.visualization.crop_image(x, top_ratio=0.15, bottom_ratio=0.2, left_ratio=0.05, right_ratio=0.02)
image_xy_slice = crop(image_xy_slice)
label_xy_slice = crop(label_xy_slice)

color_dictionary = {
    'violet': (170/255, 48/255, 127/255, 0.35),
    'blue':   (0, 0, 1, 0.35),
}
#                                         ct               liver                        spleen                      
radipop_utils.visualization.overlay_masks(image_xy_slice, [label_xy_slice==5,           label_xy_slice==1], 
                                                          [color_dictionary['violet'],  color_dictionary['blue']]);
radipop_utils.visualization.overlay_masks(image_xy_slice, [], []);

In [ ]:
img_array.shape

In [ ]:
print("done")